In [1]:
import random
import math
import copy

In [4]:
class Test(CRO):
    """
    This is a derived class to find the minimum value of sin(x) + cos(y).
    In real terms, you should implement this class according to your real problem.
    Note that every copy operation is deep-copy.
    """
    def __init__(self, structure):
        CRO.__init__(self, structure)

    def wall(self, m):
        new = copy.deepcopy(m)
        new.structure[0], new.structure[1] = new.structure[1], new.structure[0]
        return new

    def dec(self, m):
        new1 = copy.deepcopy(m)
        new2 = copy.deepcopy(m)
        new1.structure[0] += random.random()
        new2.structure[1] += random.random()
        return [new1, new2]

    def inter(self, m1, m2):
        new1 = copy.deepcopy(m1)
        new2 = copy.deepcopy(m2)
        new1.structure[0] = m2.structure[0]
        new1.structure[1] = m1.structure[1]
        new2.structure[0] = m1.structure[0]
        new2.structure[1] = m2.structure[1]
        return [new1, new2]

    def syn(self, m1, m2):
        new = copy.deepcopy(m1)
        if random.random() < 0.5:
            new.structure[0] = m2.structure[0]
        else:
            new.structure[1] = m2.structure[1]
        return new

    def fit_func(self, m):
            return math.sin(m.structure[0]) + math.cos(m.structure[1])

In [3]:
class Molecule:
    def __init__(self, structure):
        self.pe = 0
        self.ke = 0
        self.opt = 9999999
        self.num_of_hits = 0
        self.structure = copy.deepcopy(structure)

    def update(self):
        """
        If this molecule has a lower energy, reset num_of_hits.
        """
        if self.pe < self.opt:
            self.opt = self.pe
            self.num_of_hits = 0
    def __str__(self):
        reVal = ''
        for i in self.structure:
            reVal+=str(i)+" "
        return reVal


class CRO:
    """
    A base class represent the CRO algorithm.
    You should inherit from this class.
    """
    optimal = None  # A Molecule object represent the optimal solution
    cnt = 0
    limit = 1000
    KELossRate = 0.2
    mole_coll = 0.2
    alpha = 100
    beta = 100
    buffer = 0
    init_ke = 100
    pop = []

    def __init__(self, structure):
        """
        * fit_func: Object function
        * structure: Initial solution list [s1, s2, ..., sn]
        """
        for s in structure:
            self.pop.append(Molecule(s))

        for mol in self.pop:
            # You should implement this function in your derived class
            mol.pe = self.fit_func(mol)
            mol.ke = self.init_ke
            mol.update()
            if self.optimal is None:
                self.optimal = copy.deepcopy(mol)
            elif mol.pe < self.optimal.pe:
                self.optimal = copy.deepcopy(mol)

    def run(self):
        while self.cnt < self.limit:
            self.cnt += 1
            if random.random() > self.mole_coll or len(self.pop) < 2:
                m = random.choice(self.pop)
                if m.num_of_hits > self.alpha:
                    self.decomposition(m)
                else:
                    self.on_wall(m)
            else:
                m1, m2 = random.sample(self.pop, 2)
                if m1.ke <= self.beta and m2.ke <= self.beta:
                    self.synthesis(m1, m2)
                else:
                    self.interaction(m1, m2)

    def update(self, m):
        """
        If m is the current optimal solution, save it to the optimal.
        """
        if m.pe < self.optimal.pe:
            self.optimal = copy.deepcopy(m)

    def decomposition(self, m):
        m.num_of_hits += 1

        # You should implement this function in your derived class
        new1, new2 = self.dec(m)
        new1.pe = self.fit_func(new1)
        new2.pe = self.fit_func(new2)
        tmp = m.pe + m.ke - new1.pe - new2.pe
        if tmp >= 0 or tmp + self.buffer >= 0:
            if tmp >= 0:
                q = random.random()
                new1.ke = tmp * q
                new2.ke = tmp * (1 - q)
            else:
                new1.ke = (tmp + self.buffer) * random.random() * random.random()
                new2.ke = (tmp + self.buffer - new1.ke) * random.random() * random.random()
                self.buffer = self.buffer + tmp - new1.ke - new2.ke
            new1.update()
            new2.update()
            self.pop.remove(m)
            self.pop.append(new1)
            self.pop.append(new2)
            self.update(new1)
            self.update(new2)

    def on_wall(self, m):
        m.num_of_hits += 1

        # You should implement this function in your derived class
        new = self.wall(m)
        new.pe = self.fit_func(new)
        tmp = m.pe + m.ke - new.pe
        if tmp >= 0:
            q = random.uniform(self.KELossRate, 1)
            new.ke = tmp * q
            new.update()
            self.buffer += tmp * (1 - q)
            self.pop.remove(m)
            self.pop.append(new)
            self.update(new)

    def interaction(self, m1, m2):
        m1.num_of_hits += 1
        m2.num_of_hits += 1

        # You should implement this function in your derived class
        new1, new2 = self.inter(m1, m2)
        new1.pe = self.fit_func(new1)
        new2.pe = self.fit_func(new2)
        tmp = m1.pe + m2.pe + m1.ke + m2.ke - new1.pe - new2.pe
        if tmp >= 0:
            q = random.random()
            new1.ke = tmp * q
            new2.ke = tmp * (1 - q)
            new1.update()
            new2.update()
            self.pop.remove(m1)
            self.pop.remove(m2)
            self.pop.append(new1)
            self.pop.append(new2)
            self.update(new1)
            self.update(new2)

    def synthesis(self, m1, m2):
        m1.num_of_hits += 1
        m2.num_of_hits += 1

        # You should implement this function in your derived class
        new = self.syn(m1, m2)
        new.pe = self.fit_func(new)
        tmp = m1.pe + m2.pe + m1.ke + m2.ke - new.pe
        if tmp >= 0:
            new.ke = tmp
            new.update()
            self.pop.remove(m1)
            self.pop.remove(m2)
            self.pop.append(new)
            self.update(new)

In [5]:
s = [[random.random(), random.random()] for i in range(10)]
print(s)
cro = Test(s)
cro.run()
print(cro.optimal.pe)

[[0.2692380459886048, 0.8360274776068701], [0.10051601196989401, 0.9180841602082337], [0.39677835924523397, 0.9880215878733322], [0.9983175814897175, 0.0632059297456552], [0.4958385537761413, 0.19586159921468627], [0.6531845218076264, 0.9874749343293858], [0.7575369892491454, 0.6885212143003785], [0.8986502934824694, 0.06319741058085993], [0.8266892224821328, 0.45559793023125417], [0.09131973862533138, 0.7961762908319385]]
-1.9388500188274387


In [ ]:
import random

# Define the function to generate supersequences
def generate_supersequences(input_strings, pop_size, superseq_length):
    population = []

    for _ in range(pop_size):
        superseq = ['-' for _ in range(superseq_length)]  # Start with an empty supersequence
        for string in input_strings:
            positions = sorted(random.sample(range(superseq_length), len(string)))  # Random unique positions
            for pos, char in zip(positions, string):
                superseq[pos] = char  # Place char in the supersequence
        
        # Convert the list of characters back to a string, replacing empty slots with random picks
        superseq = ''.join(char if char != '-' else random.choice('acgt') for char in superseq)
        population.append(superseq)
    
    return population

# Example usage:
input_strings = ['acg', 'cat', 'gtt']
pop_size = 20
superseq_length = 6  # Twice the length of the longest string

# Generate the population
initial_population = generate_supersequences(input_strings, pop_size, superseq_length)
initial_population  # Display the generated population


['gtcgtt',
 'tagttg',
 'tgttat',
 'acgatt',
 'gtagtt',
 'cagttt',
 'agctgt',
 'gcttcg',
 'tagttt',
 'gattat',
 'ggtctt',
 'gtctat',
 'tgtatt',
 'gtagtt',
 'gacatt',
 'gcattt',
 'cgctgt',
 'cagttt',
 'cggttg',
 'cgtttt']

# MY IMCRO

In [ ]:
# Initialization


In [ ]:
class MoleCule:
    def __init__(self, structure,supersequence):
        self.pe = 0
        self.ke = 0
        self.opt = 9999999
        self.num_of_hits = 0
        self.structure = copy.deepcopy(structure)
        self.supersequence = copy.deepcopy(supersequence)

    def update(self):
        """
        If this molecule has a lower energy, reset num_of_hits.
        """
        if self.pe < self.opt:
            self.opt = self.pe
            self.num_of_hits = 0
    def __str__(self):
        reVal = ''
        for i in self.structure:
            reVal+=str(i)+" "
        return reVal

In [ ]:
def uni_moleReact():
    return
def inter_moleReact():
    return

In [ ]:
# Iteration
i = 0
moleColl = 0.2
alpha = random.randint(10, 100)
beta = random.randint(10, 100)
while i!=1000:
    i+=1
    t = random.random()
    if t > moleColl:
        uni_moleReact()
    else :
        inter_moleReact()
    

In [ ]:
import random

# Function to generate molecules with PE and KE
def generate_molecules(num_molecules, pe_range, ke_range, pe_ke_ratio):
    """
    Generates a list of molecules with random Potential Energy (PE) and Kinetic Energy (KE).
    The PE and KE values are generated with a constraint to keep them not too distinguishable.
    
    :param num_molecules: Number of molecules to generate.
    :param pe_range: Tuple (min, max) defining the range for PE.
    :param ke_range: Tuple (min, max) defining the range for KE.
    :param pe_ke_ratio: The ratio to ensure PE and KE are not too distinguishable.
    :return: List of dictionaries containing PE and KE for each molecule.
    """
    molecules = []
    for _ in range(num_molecules):
        # Generate PE and KE based on given ranges and ensure they are not too distinguishable using the ratio
        pe = random.uniform(*pe_range)
        # KE is constrained around PE value within a range modified by pe_ke_ratio
        ke = random.uniform(pe * pe_ke_ratio, pe * pe_ke_ratio + (ke_range[1] - ke_range[0]))
        molecules.append({'PE': pe, 'KE': ke})
    
    return molecules

# Example usage:
# Generate 20 molecules with PE between 10 and 20 and KE between 5 and 15, with a ratio of 0.8 to 1.2 for PE to KE
num_molecules = 20
pe_range = (10, 100)
ke_range = (10, 100)
pe_ke_ratio = 0.8  # Ratio to ensure PE and KE are closely matched

molecules = generate_molecules(num_molecules, pe_range, ke_range, pe_ke_ratio)
molecules  # Display the generated molecules with their PE and KE values


[{'PE': 99.5640192565935, 'KE': 104.45842538021843},
 {'PE': 18.48879226910879, 'KE': 63.380838614175076},
 {'PE': 55.63908620355706, 'KE': 128.93384054206635},
 {'PE': 42.960546885027455, 'KE': 90.48050981733456},
 {'PE': 79.0586403737632, 'KE': 69.32365182814856},
 {'PE': 51.999548377644906, 'KE': 113.58010147151066},
 {'PE': 33.64775112107074, 'KE': 98.1491636371076},
 {'PE': 60.29871849848696, 'KE': 118.65712580291964},
 {'PE': 31.792285307246328, 'KE': 50.67620971681937},
 {'PE': 75.93399946315857, 'KE': 75.6534089806905},
 {'PE': 96.81699119133873, 'KE': 138.06961979011191},
 {'PE': 34.49173325851888, 'KE': 112.37132751716999},
 {'PE': 11.793680950906218, 'KE': 19.921829333170365},
 {'PE': 14.272506475104615, 'KE': 38.53209057243764},
 {'PE': 58.230462731870816, 'KE': 109.05973093639372},
 {'PE': 76.57304140934582, 'KE': 66.83483296432941},
 {'PE': 81.79895814006449, 'KE': 92.103380895775},
 {'PE': 65.47966954187294, 'KE': 133.50042342899155},
 {'PE': 95.86274361623367, 'KE': 110